# --- Day 6: Universal Orbit Map ---

## ---Part One---

You've landed at the Universal Orbit Map facility on Mercury. Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).

Except for the universal Center of Mass (COM), every object in space is in <a src="https://en.wikipedia.org/wiki/Orbit">orbit</a> around exactly one other object. An orbit looks roughly like this:
```

                 \
                   \
                    |
                    |
AAA--> o            o <--BBB
                    |
                    |
                   /
                  /
```

In this diagram, the object ```BBB``` is in orbit around ```AAA```. The path that ```BBB``` takes around ```AAA``` (drawn with lines) is only partly shown. In the map data, **this orbital relationship is written ```AAA)BBB```, which means "```BBB``` is in orbit around ```AAA```"**.

Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download. To verify maps, the Universal Orbit Map facility uses orbit count checksums:
* the total number of direct orbits (like the one shown above)
* indirect orbits.

Whenever ```A``` orbits ```B``` and ```B``` orbits ```C```, then ```A``` indirectly orbits ```C```. This chain can be any number of objects long: if ```A``` orbits ```B```, ```B``` orbits ```C```, and ```C``` orbits ```D```, then ```A``` indirectly orbits ```D```.

**For example, suppose you have the following map:**

> `COM)B
 B)C
 C)D
 D)E
 E)F
 B)G
 G)H
 D)I
 E)J
 J)K
 K)L`

**Visually, the above map of orbits looks like this:**

```
         G - H       J - K - L
        /           /
 COM - B - C - D - E - F
                \
                 I
```

In this visual representation, when two objects are connected by a line, the one on the right directly orbits the one on the left.

**Here, we can count the total number of orbits as follows:**

>* `D` directly orbits `C` and indirectly orbits `B` and `COM`, a total of `3` orbits.
>* `L` directly orbits `K` and indirectly orbits `J`, `E`, `D`, `C`, `B`, and `COM`, a total of `7` orbits.
>* `COM` orbits nothing.
>* The total number of direct and indirect orbits in this example is `42`.

**What is the total number of direct and indirect orbits in your map data?**

In [1]:
def get_direct(map_data):
    '''
    key is in orbit aroung value
    '''
    return { value : key for key, value in map_data }

In [2]:
ex1 = ['COM)B', 'B)C', 'C)D', 'D)E', 'E)F', 'B)G', 'G)H', 'D)I', 'E)J', 'J)K', 'K)L']
ex1_orbits = [ i.strip().split(')') for i in ex1 ]
direct = get_direct(ex1_orbits)
direct

In [3]:
def planet_orbits(direct, planet, destination="COM"):
    if planet == destination:
        return 0
    elif planet == 'COM':
        print("You've reached the COM, your destination was not found, try switching the order of your planet and destination")
        return float('inf')
    else:
        return 1 + planet_orbits(direct, direct[planet], destination)

In [4]:
print( f'D: {planet_orbits(direct,"D")} orbits' )
print( f'L: {planet_orbits(direct,"L")} orbits' )
print( f'COM: {planet_orbits(direct,"COM")} orbits' )

D: 3 orbits
L: 7 orbits
COM: 0 orbits


In [5]:
planet_orbits(direct,"D",destination="K")

You've reached the COM, your destination was not found, try switching the order of your planet and destination


inf

In [6]:
def system_orbits(map_data):
    direct = get_direct(map_data)
    planets = [ planet_orbits(direct, planet) for planet in direct ]
    return sum(planets)

In [7]:
system_orbits(ex1_orbits)

42

In [8]:
with open('day6.txt') as f:
    map_data = [ line.strip().split(')') for line in f ]
    print(f'The system has {system_orbits(map_data)} orbits.')

The system has 119831 orbits.


## ---Part Two---

Now, you just need to figure out how many orbital transfers you (```YOU```) need to take to get to Santa (```SAN```).

You start at the object YOU are orbiting; your destination is the object ```SAN``` is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.

**For example, suppose you have the following map:**

```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
```

**Visually, the above map of orbits looks like this:**

```
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
```

In this example, ```YOU``` are in orbit around ```K```, and ```SAN``` is in orbit around ```I```. To move from ```K``` to ```I```, a minimum of 4 orbital transfers are required:

* ```K``` to ```J```
* ```J``` to ```E```
* ```E``` to ```D```
* ```D``` to ```I```

**Afterward, the map of orbits looks like this:**

```
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
                 \
                  YOU
```

**What is the minimum number of orbital transfers required to move from the object ```YOU``` are orbiting to the object ```SAN``` is orbiting (Between the objects they are orbiting - not between ```YOU``` and ```SAN```)?**

In [9]:
def get_transfers(direct, planet, destination="COM"):
    visited = set()
    if type(planet_orbits(direct,planet,destination)) == int:
        while direct[planet] != destination:
            visited.add(direct[planet])
            planet = direct[planet]
        visited.add(direct[planet])
        return visited
    else:
        print("Path not found from you to destination, try reversing their order.")

In [10]:
ex2 = [ 'COM)B', 'B)C', 'C)D', 'D)E', 'E)F', 'B)G', 'G)H', 'D)I', 'E)J', 'J)K', 'K)L', 'I)SAN', 'K)YOU' ]
ex2_orbits = [ i.strip().split(')') for i in ex2 ]
direct = get_direct(ex2_orbits)

you = get_transfers(direct, "YOU")
san = get_transfers(direct, "SAN")
print( f'Number of Transfers: {len(( you - san )| ( san - you ))}')
print( f'Planets Visited: {( you - san ) | ( san - you )}')

Number of Transfers: 4
Planets Visited: {'K', 'I', 'J', 'E'}


***Quick note here: I learned (or was reminded from this) that set complement (set theory) is not the same as set difference in python. The set difference in python only gives you the items in set A that aren't in set B. In order to get the full complement you must union the difference of A and B with the difference of B and A.***

In [11]:
with open('day6.txt') as f:
    map_data = [ line.strip().split(')') for line in f ]
    direct = get_direct(map_data)
    paths_to_com = [ get_transfers(direct, i) for i in ["YOU", "SAN"] ]
    path_to_santa = ( paths_to_com[0] - paths_to_com[1] ) | ( paths_to_com[1] - paths_to_com[0] )
    
    print(f"YOU are {planet_orbits(direct,'YOU')} transfers from COM")
    print(f"SANTA is {planet_orbits(direct,'SAN')} transfers from COM")
    
    print(f"YOU are {len(path_to_santa)} transfers away from SANTA")

YOU are 248 transfers from COM
SANTA is 208 transfers from COM
YOU are 322 transfers away from SANTA
